In [1]:
import pandas as pd
import sqlite3
pd.set_option('display.max_columns', None)

In [ ]:
conn=sqlite3.connect('test_db.db')

In [2]:
df = pd.read_csv('uploads/Crime_01_2024.csv')
df

,IncidentDate,OccurredFromTime,IncidentNum,Offense,NIBRS,NIBRSCategory,SRS_UCR,CrimeAgainst,FelMisdCit,IncidentTopSRS_UCR,IncidentLocation,IntersectionOtherLoc,District,Neighborhood,NbhdNum,Latitude,Longitude,IncidentSupplemented,LastSuppDate,VictimNum,FirearmUsed,IncidentNature
0,2020-12-17,18:00:00,20056840,STEALING - $750 OR MORE,23H,All Other Larceny,6.0,Property,F,6.0,8025 S BROADWAY,NaN,1.0,Patch,2,38.544389,-90.262290,Yes,2024-01-29 08:55:56.3400000,NaN,No,Stolen Property - Criminal
1,2020-12-17,18:00:00,20056840,PROPERTY DAMAGE 1ST DEGREE,290,Destruction/Damage/Vandalism of Property,NaN,Property,NaN,6.0,8025 S BROADWAY,NaN,1.0,Patch,2,38.544389,-90.262290,Yes,2024-01-29 08:55:56.3400000,NaN,No,Stolen Property - Criminal
2,2020-12-23,06:05:00,20057591,ROBBERY - 1ST DEGREE,120,Robbery,3.0,Property,NaN,3.0,2600 N 14TH ST,NaN,4.0,Old North St Louis,63,38.650037,-90.196741,Yes,2024-01-27 00:05:05.7070000,NaN,Yes,Carjacking - Criminal
3,2020-12-29,00:52:00,20058325,ROBBERY - 1ST DEGREE,120,Robbery,3.0,Property,NaN,3.0,4922 CLAXTON AVE,NaN,6.0,Mark Twain,71,38.689941,-90.249730,Yes,2024-01-12 13:47:31.0000000,NaN,Yes,Robbery - Criminal
4,2020-12-31,22:20:00,20058746,ASSAULT 1ST DEGREE OR ATTEMPT,13A,Aggravated Assault,4.0,Person,NaN,4.0,811 SPRUCE ST,ROOM 311,4.0,Downtown,35,38.623713,-90.194768,Yes,2024-01-08 08:33:09.0300000,363565.0,Yes,Aggravated Assault - Shooting - Criminal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,2024-01-25,12:00:00,24007184,STEALING UNDER $150 (PETTY LARCENY - OTHER),23H,All Other Larceny,6.0,Property,NaN,6.0,5118 WATERMAN BLVD,NaN,5.0,Central West End,38,38.648258,-90.268370,No,NaN,NaN,No,Larceny - Criminal
5871,2024-01-31,00:01:00,24007205,DESTRUCTION OF PRIVATE PROPERTY (CITY CHARGE),290,Destruction/Damage/Vandalism of Property,NaN,Property,NaN,NaN,7208 N BROADWAY,NaN,6.0,North Riverfront,79,38.631053,-90.187720,No,NaN,NaN,No,Destruction of Property - Criminal
5872,2024-01-26,11:00:00,24007385,STEALING,23D,Theft From Building,6.0,Property,F,6.0,4021 LACLEDE AVENUE,NaN,5.0,Central West End,38,38.637080,-90.244946,No,NaN,NaN,No,Larceny - Criminal
5873,2024-01-22,12:00:00,24007544,LEAVING THE SCENE OF A MOTOR VEHICLE ACCIDENT,90Z,All Other Offenses,NaN,Unspecified,NaN,NaN,6014 MARMADUKE AVE,NaN,2.0,Clifton Heights,11,38.609736,-90.290430,No,NaN,NaN,No,Offenses - All Other - Criminal


In [3]:
# df['Id'] = None
df = df.drop('IncidentTopSRS_UCR', axis=1)
df.rename(columns={'CrimeAgainst': 'NIBRSCat',
                   'NIBRS': 'NIBRSCode',
                   'NIBRSCategory':'NIBRSOffenseType',
                   'SRS_UCR':'UCR_SRS',
                   'OccurredFromTime':'TimeOccurred',
                   'Offense':'SLMPDOffense',
                   'FelMisdCit':'CrimeGrade',
                   'IncidentLocation':'PrimaryLocation',
                   'IntersectionOtherLoc':'SecondaryLocation',
                   'NbhdNum':'NeighborhoodNum',
                   'IncidentSupplemented':'Supplemented',
                   'LastSuppDate':'SupplementDate'}, inplace=True)

ordered_cols = ['IncidentNum', 'IncidentDate', 'TimeOccurred', 'SLMPDOffense',
                'NIBRSCode', 'NIBRSCat', 'NIBRSOffenseType', 'UCR_SRS', 'CrimeGrade',
                'PrimaryLocation', 'SecondaryLocation', 'District', 'Neighborhood',
                'NeighborhoodNum', 'Latitude', 'Longitude', 'Supplemented',
                'SupplementDate', 'VictimNum', 'FirearmUsed', 'IncidentNature']
df = df[ordered_cols]

# remove incidents prior to 2021-01-01
df['IncidentDate'] = pd.to_datetime(df['IncidentDate'])
df = df[~(df['IncidentDate'] < '2021-01-01')]
# revert to sting column
df.loc[:,'IncidentDate'] = df.loc[:,'IncidentDate'].astype('str')


supp_df = df[df['Supplemented'] == 'Yes']
unfound_df = df[(df['Supplemented'].isna()) & (df['SLMPDOffense'] == 'UNFOUNDED INCIDENT')]
new_df = df[df['Supplemented'] == 'No']

# new_rows = supp_df.itertuples(index=False, name=None)
# new_rows = supp_df.itertuples(index=False)
new_rows = [tuple(x) for x in supp_df.itertuples(index=False)]

In [12]:
supp_df.sort_values('IncidentNum')
# supp_df.sort_values('IncidentDate', ascending=True)

,IncidentNum,IncidentDate,TimeOccurred,SLMPDOffense,NIBRSCode,NIBRSCat,NIBRSOffenseType,UCR_SRS,CrimeGrade,PrimaryLocation,SecondaryLocation,District,Neighborhood,NeighborhoodNum,Latitude,Longitude,Supplemented,SupplementDate,VictimNum,FirearmUsed,IncidentNature
5,21000522,2021-01-04,12:50:00,MURDER 1ST DEGREE,09A,Person,Murder and Nonnegligent Manslaughter,1.0,NaN,4036 ARSENAL ST,1ST FLOOR,2.0,Tower Grove South,15,38.603613,-90.253747,Yes,2024-01-19 08:52:48.4800000,366839.0,Yes,Homicide - Criminal
6,21000790,2021-01-06,08:00:00,VIOLATION OF ORDER OF PROTECTION FOR ADULT - 1...,90Z,Unspecified,All Other Offenses,NaN,NaN,620 S TAYLOR AVE,NaN,5.0,Central West End,38,38.634742,-90.258980,Yes,2024-01-17 19:48:29.7230000,NaN,No,Offenses - All Other - Criminal
7,21000840,2021-01-06,12:00:00,MURDER 1ST DEGREE,09A,Person,Murder and Nonnegligent Manslaughter,1.0,NaN,4481 PENROSE,NaN,6.0,Penrose,69,38.676546,-90.235020,Yes,2024-01-22 11:51:21.7130000,367426.0,Yes,Aggravated Assault - Shooting - Criminal
8,21000905,2021-01-06,17:00:00,ASSAULT 1ST DEGREE OR ATTEMPT,13A,Person,Aggravated Assault,4.0,NaN,LILLIAN AVE AND THRUSH AVE,NaN,6.0,Walnut Park East,72,38.695750,-90.253520,Yes,2024-01-22 11:09:43.3530000,367401.0,Yes,Aggravated Assault - Shooting - Criminal
9,21000908,2021-01-06,19:14:00,ASSAULT 1ST DEGREE OR ATTEMPT - SERIOUS PHYSIC...,13A,Person,Aggravated Assault,4.0,NaN,UTAH ST,NaN,3.0,Marine Villa,18,38.595299,-90.226944,Yes,2024-01-22 10:27:05.5130000,367377.0,Yes,Aggravated Assault - Shooting - Criminal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5853,24005524,2024-01-12,06:00:00,STEALING - ALL OTHER PROPERTY UNDER 570.030.5 (3),23D,Property,Theft From Building,6.0,F,3937 N 20TH ST,NaN,4.0,Hyde Park,65,38.663273,-90.204579,Yes,2024-02-15 08:31:49.0200000,NaN,No,Larceny - Criminal
5856,24005689,2024-01-27,11:05:00,STEALING - POSTAL OR DELIVERY ITEM,23H,Property,All Other Larceny,6.0,NaN,59 WILLMORE RD,NaN,2.0,St Louis Hills,8,38.577052,-90.307070,Yes,2024-02-21 08:35:04.7330000,NaN,No,Larceny - Criminal
5860,24006233,2024-01-25,10:14:00,STEALING - MOTOR VEHICLE/WATERCRAFT/AIRCRAFT &...,26A,Property,False Pretense/Swindle/Confidence Game,NaN,F,2210 S 7TH ST,NaN,3.0,Kosciusko,20,38.604615,-90.203040,Yes,2024-02-14 08:08:26.0000000,NaN,No,Fraud - Criminal
5865,24006728,2024-01-09,06:00:00,STEALING - $750 OR MORE,23H,Property,All Other Larceny,6.0,F,1700 WASHINGTON AVE,NaN,4.0,Downtown West,36,38.633288,-90.203490,Yes,2024-02-24 18:50:20.4800000,NaN,No,Larceny - Criminal


In [16]:
supp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3844 entries, 5 to 5867
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   IncidentNum        3844 non-null   int64  
 1   IncidentDate       3844 non-null   object 
 2   TimeOccurred       3844 non-null   object 
 3   SLMPDOffense       3844 non-null   object 
 4   NIBRSCode          3844 non-null   object 
 5   NIBRSCat           3844 non-null   object 
 6   NIBRSOffenseType   3844 non-null   object 
 7   UCR_SRS            1507 non-null   float64
 8   CrimeGrade         3025 non-null   object 
 9   PrimaryLocation    3844 non-null   object 
 10  SecondaryLocation  595 non-null    object 
 11  District           3844 non-null   float64
 12  Neighborhood       3844 non-null   object 
 13  NeighborhoodNum    3844 non-null   object 
 14  Latitude           3831 non-null   float64
 15  Longitude          3831 non-null   float64
 16  Supplemented       3844 non-n

In [15]:
supp_df['IncidentDate'] = supp_df['IncidentDate'].astype('str')

C:\Users\diper\AppData\Local\Temp\ipykernel_10588\1956342329.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  supp_df['IncidentDate'] = supp_df['IncidentDate'].astype('str')


In [4]:
list(new_rows)

[(21000522,
  Timestamp('2021-01-04 00:00:00'),
  '12:50:00',
  'MURDER 1ST DEGREE',
  '09A',
  'Person',
  'Murder and Nonnegligent Manslaughter',
  1.0,
  nan,
  '4036 ARSENAL ST',
  '1ST FLOOR',
  2.0,
  'Tower Grove South',
  '15',
  38.603613,
  -90.253747,
  'Yes',
  '2024-01-19 08:52:48.4800000',
  366839.0,
  'Yes',
  'Homicide - Criminal'),
 (21000790,
  Timestamp('2021-01-06 00:00:00'),
  '08:00:00',
  'VIOLATION OF ORDER OF PROTECTION FOR ADULT - 1ST OFFENSE',
  '90Z',
  'Unspecified',
  'All Other Offenses',
  nan,
  nan,
  '620 S TAYLOR AVE',
  nan,
  5.0,
  'Central West End',
  '38',
  38.634742,
  -90.25898,
  'Yes',
  '2024-01-17 19:48:29.7230000',
  nan,
  'No',
  'Offenses - All Other - Criminal'),
 (21000840,
  Timestamp('2021-01-06 00:00:00'),
  '12:00:00',
  'MURDER 1ST DEGREE',
  '09A',
  'Person',
  'Murder and Nonnegligent Manslaughter',
  1.0,
  nan,
  '4481 PENROSE',
  nan,
  6.0,
  'Penrose',
  '69',
  38.676546,
  -90.23502,
  'Yes',
  '2024-01-22 11:51:21.

In [ ]:
jan['IncidentDate'] = pd.to_datetime(jan['IncidentDate'])

res = jan[~(jan['IncidentDate'] < '2021-01-01')]
res

In [ ]:
list(res.itertuples(index=False, name=None))

In [ ]:
res.loc[:,'IncidentDate'] = res.loc[:,'IncidentDate'].astype('str')

In [ ]:
res.info()

In [ ]:
sql_df = pd.read_sql_query("SELECT * FROM crime_data", conn)

In [ ]:
sql_df

In [ ]:
overlap = jan[jan['IncidentNum'].isin(sql_df['IncidentNum'])]

In [ ]:
overlap['IncidentNum'].value_counts()

In [ ]:
overlap[overlap['IncidentNum'] == 23005581]

In [ ]:
sql_df[sql_df['IncidentNum'] == 23005581]